In [10]:
import os
import zipfile
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [11]:
print(len(os.listdir('data/training/hitsuji/')))
print(len(os.listdir('data/training/inu/')))
print(len(os.listdir('data/training/ne/')))
print(len(os.listdir('data/training/tori/')))
print(len(os.listdir('data/training/uma/')))
print(len(os.listdir('data/validation/hitsuji/')))
print(len(os.listdir('data/validation/inu/')))
print(len(os.listdir('data/validation/ne/')))
print(len(os.listdir('data/validation/tori/')))
print(len(os.listdir('data/validation/uma/')))

450
450
450
450
450
50
50
50
50
50


In [16]:
model = tf.keras.models.Sequential([
# YOUR CODE HERE
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['acc'])

In [17]:
TRAINING_DIR = "data/training"
VALIDATION_DIR = "data/validation"
train_datagen = ImageDataGenerator( 
    rescale = 1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(300, 300))
validation_generator = test_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical')


Found 2250 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [18]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [19]:
#model.load_weights(checkpoint_path)
history = model.fit_generator(train_generator,
                               epochs=13,
                               validation_data=validation_generator,
                               verbose=1,
                               callbacks=[cp_callback])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 113 steps, validate for 13 steps
Epoch 1/13
112/113 [============================>.] - ETA: 0s - loss: 1.7904 - acc: 0.3058
Epoch 00001: saving model to training_1/cp.ckpt
113/113 [==============================] - 67s 591ms/step - loss: 1.7841 - acc: 0.3071 - val_loss: 1.0569 - val_acc: 0.4720
Epoch 2/13
112/113 [============================>.] - ETA: 0s - loss: 0.9747 - acc: 0.6291
Epoch 00002: saving model to training_1/cp.ckpt
113/113 [==============================] - 69s 607ms/step - loss: 0.9707 - acc: 0.6307 - val_loss: 0.2290 - val_acc: 0.9280
Epoch 3/13
112/113 [============================>.] - ETA: 0s - loss: 0.5520 - acc: 0.8179
Epoch 00003: saving model to training_1/cp.ckpt
113/113 [==============================] - 67s 596ms/step - loss: 0.5486 - acc: 0.8187 - val_loss: 0.0449 - val_acc: 0.9960
Epoch 4/13
112/113 [============================>.] - ETA: 0s - loss: 0.3759 - acc: 0.8758
Epoch 00004: saving model t

In [20]:
def findMax(list):
    result = 0
    id_result = 0
    for i in range(len(list)):
        if list[i] > result:
            result = list[i]
            id_result = i
    
    return os.listdir(TRAINING_DIR)[id_result]

In [21]:
# pred_dir='test'
# test_generator = test_datagen.flow_from_directory(
#     directory=pred_dir,
#     target_size=(300, 300),
#     color_mode="rgb",
#     batch_size=20,
#     class_mode='categorical',
#     shuffle=False
# )
# test_generator.reset()

# pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

# print(findMax(pred[0]))

In [26]:
import cv2
import time
import os
from tensorflow.keras.preprocessing import image
class TakeImage:
    def __init__(self, directory, side, num_img, duration=2):
        self.directory = directory              # folder name
        self.side = side                        # side of the image sidexside
        if not os.path.isdir(directory):        # check if the directory exits or not
            os.mkdir(directory)                 # if not, create one    
        self.count = len(os.listdir(directory)) # number of images currently in the directory
        self.number = num_img                   # number of images need to take
        self.duration = duration                # time between images

    def take(self):
        # create Video Capture
        key = cv2.waitKey(1)
        camera = cv2.VideoCapture(0, cv2.CAP_DSHOW)

        start = time.time()
        pause = False
        result = 0
        while True:
            try:
                # get a frame from camera
                check, frame = camera.read()
                # draw a cropped rectangle
                start_x, start_y, end_x, end_y = self.getRectanglePoints(
                    camera.get(cv2.CAP_PROP_FRAME_WIDTH),
                    camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
                # -2 because we don't want to get the border of the rectangle in images
                cv2.rectangle(frame, (start_x-2, start_y-2), (end_x, end_y),(0,255,0), 1)

                if not pause:
                # add timer
                    timer = self.timer(start)

                cv2.putText(frame, str(result),(10, 105),cv2.FONT_HERSHEY_SIMPLEX,2,(225,255,255),3)

                # show frame
                cv2.imshow("Capturing", frame)
                key = cv2.waitKey(1)

                # exit if the folder have all image or user hit the s key
                if self.count == self.number or key == ord('s'): 
                    camera.release()  
                    print("You have taken "+str(self.count)+" images "+" in "+self.directory)
                    break

                if key == ord('p'):
                    pause = True


                if key == ord('c'):
                    start = time.time()
                    pause = False

                # save image
                if timer == 0 and not pause:
                    #crop the frame
                    crop = frame[start_y:start_y+self.side,start_x:start_x+self.side]

                    # save image
                    image_name = self.getNextName()
                    path='test/all/test.jpg'
                    if cv2.imwrite(filename=path, img=crop):
                        print(path+" is saved in "+self.directory)
                    else:
                        print("Fail to save "+image_name)

                    
                    pred_dir='test'
                    test_generator = test_datagen.flow_from_directory(
                        directory=pred_dir,
                        target_size=(300, 300),
                        color_mode="rgb",
                        batch_size=20,
                        class_mode='categorical',
                        shuffle=False
                    )
                    test_generator.reset()

                    pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
                    print(pred[0])
                    result = findMax(pred[0])
#                     if findMax(pred[0]) == 'zero':
#                         result = 0
#                     if findMax(pred[0]) == 'one':
#                         result = 1
#                     if findMax(pred[0]) == 'two':
#                         result = 2
                    
                    #os.remove(path)
                    self.count += 1
                    start = time.time()

            except(KeyboardInterrupt):
                print("Turning off camera.")
                camera.release()
                print("Camera off.")
                print("Program ended.")
                cv2.destroyAllWindows()
                break

    def getRectanglePoints(self, width, height):
        # initialize start point and end point
        start_x = start_y = 0
        end_x = end_y = 0
        # 
        start_x = int(width/2 - self.side/2)
        start_y = int(height/2 - self.side/2)
        end_x = int(width/2 + self.side/2)
        end_y = int(height/2 + self.side/2)
        return start_x, start_y, end_x, end_y

    def timer(self, start):
        end = time.time()
        return int(self.duration - end + start)+1

    def getNextName(self):
        name = ""
        images = os.listdir(self.directory)
        if len(images) == 0:
            name = self.directory+"-"+str(0)+".jpg"
        else:
            num = len(images)
            name = self.directory+"-"+str(num)+".jpg"
        return name

In [27]:
test = TakeImage("test", 300, 100,1)
test.take()

test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[8.9234596e-01 6.0784492e-05 9.1191009e-03 9.5032610e-02 3.4415862e-03]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[4.5393845e-03 9.6211458e-08 3.3692509e-04 9.9344301e-01 1.6805842e-03]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[1.0305119e-03 2.0220899e-07 4.3558909e-04 9.9850982e-01 2.3785997e-05]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 14ms/step
[2.6923586e-02 1.3953172e-05 1.0099739e-02 9.4554853e-01 1.7414210e-02]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 16ms/step
[7.3759304e-04 4.3141415e-09 7.4194836e-06 9.9925476e-01 2.4966837e-07]
test/all/t

[7.7875865e-01 2.6329761e-10 8.1762728e-06 2.2120935e-01 2.3864884e-05]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[8.4797335e-01 1.9598079e-09 1.9916410e-05 1.5195774e-01 4.9067145e-05]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[9.0643477e-01 2.8088818e-09 3.4628083e-05 9.3447633e-02 8.3040104e-05]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[5.4387248e-01 5.9932290e-09 3.0946831e-05 4.5587811e-01 2.1846147e-04]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[5.5795163e-01 4.8175548e-09 2.5618170e-05 4.4183892e-01 1.8386442e-04]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[6.7269963

test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 16ms/step
[5.3001644e-19 1.1802005e-10 2.5701805e-10 1.0778406e-21 1.0000000e+00]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 17ms/step
[1.08680387e-09 2.62678839e-07 1.08910899e-04 1.08894053e-10
 9.99890804e-01]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[2.0487728e-06 1.2630236e-12 2.2851818e-09 9.9999797e-01 2.8267909e-12]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 15ms/step
[7.4283082e-05 7.2243295e-11 2.8744091e-06 9.9992287e-01 1.1373560e-09]
test/all/test.jpg is saved in test
Found 1 images belonging to 1 classes.
1/1 [==============================] - 0s 17ms/step
[4.1569865e-01 2.1167161e-06 1.0018012e-03 5.8329737e-01 6.7553181e-08]
test